# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
### Had a crack at reading and concatenating all of the files

song_files = get_files("data/song_data")
dfs =[]
for file in song_files:
    data = pd.read_json(file, lines=True)
    dfs.append(data)
df = pd.concat(dfs, ignore_index=True)

df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
2,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
4,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969


In [7]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()
song_data[0]

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files("data/log_data")
num_files = len(log_files)
#print('{} files found in {}'.format(num_files, filepath))
    
# create log dataframe
raw_log_dfs =[]
for file in log_files:
    data = pd.read_json(file, lines=True)
    raw_log_dfs.append(data)
log_df = pd.concat(raw_log_dfs, ignore_index=True)
    
# filter by NextSong action
log_df = log_df[log_df['page']=="NextSong"]

len(log_df.index)

6820

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [9]:
log_df_ss = log_df[log_df['page']=="NextSong"]
log_df_ss.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [15]:
#log_df_ss['ts'] = pd.to_datetime(log_df_ss['ts'], unit ='ms')
t = pd.to_datetime(log_df['ts'], unit ='ms')
t.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,2018-11-30 01:12:48.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,2018-11-30 01:17:05.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,2018-11-30 01:20:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [16]:
#(start_time, hour, day, week, month, year, weekday)
time_data = [
    ('start_time', log_df_ss['ts'].dt.time), 
    ('hour', log_df_ss['ts'].dt.hour), 
    ('day', log_df_ss['ts'].dt.day), 
    ('week', log_df_ss['ts'].dt.week),
    ('month', log_df_ss['ts'].dt.month),
    ('year', log_df_ss['ts'].dt.year),
    ('weekday', log_df_ss['ts'].dt.weekday)
]
#time_data
#column_labels = ("start_time", "hour", "day", "week", "month", "year", "weekday")

In [17]:
time_df = pd.DataFrame.from_dict(dict(time_data))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,00:22:07.796000,0,30,48,11,2018,4
1,01:08:41.796000,1,30,48,11,2018,4
2,01:12:48.796000,1,30,48,11,2018,4
3,01:17:05.796000,1,30,48,11,2018,4
4,01:20:56.796000,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = log_df_ss[['userId','firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [11]:
log_df_ss[log_df_ss['sessionId'] == 818].sort_values(by=['artist'])

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
4894,3 Doors Down,Logged In,Lily,F,36,Koch,237.13914,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Here Without You,200,1542844428796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4889,AFI,Logged In,Lily,F,32,Koch,328.72444,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Affliction,200,1542843391796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
5099,Abstract Rude,Logged In,Lily,F,57,Koch,196.85832,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Nuff Fire,200,1542850491796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4880,Al Green,Logged In,Lily,F,23,Koch,133.58975,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,I Say A Little Prayer,200,1542841463796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
5113,Alliance Ethnik,Logged In,Lily,F,68,Koch,195.94404,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,SinceritÃÂ© Et Jalousie,200,1542852842796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4883,Aly & AJ,Logged In,Lily,F,26,Koch,150.59546,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Like Whoa,200,1542841970796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
5115,Aya RL,Logged In,Lily,F,70,Koch,225.43628,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Jazz,200,1542853326796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4884,Beck,Logged In,Lily,F,27,Koch,211.14730,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Girl,200,1542842120796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4892,Ben Jelen,Logged In,Lily,F,35,Koch,226.55955,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Come On (Album Version),200,1542844202796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4876,Bertine Zetlitz,Logged In,Lily,F,19,Koch,246.30812,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Snow On A Hot Day,200,1542840436796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [6]:
# session_id = 818
for index, row in log_df_ss.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    timestamp = log_df_ss['ts'].dt.time
    #print(row)
    # insert songplay record

    songplay_data = (index,timestamp[index], row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    #print(songplay_data)
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()
    
songplay_data.head()

NameError: name 'log_df_ss' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.